In [1]:
from glob import glob

dir_fp = "/mnt/arc/levlevi/nba-positions-videos-dataset/nba-plus-statvu-dataset/filtered-clip-annotations"
all_fps = glob(dir_fp + "/*/*/*.json")
bbx_map = {}

In [4]:
import ujson as json
from tqdm import tqdm
from multiprocessing import Pool
import numpy as np

def process_file(fp):
    with open(fp, 'rb') as f:
        annotations = json.load(f)
    frame_indices = []
    bbx_indices = []
    bbxes = []
    for frame_idx, frame in enumerate(annotations["frames"]):
        n_bbxes = len(frame["bbox"])
        frame_indices.extend([frame_idx] * n_bbxes)
        bbx_indices.extend(range(n_bbxes))
        bbxes.extend(frame["bbox"])
    return frame_indices, bbx_indices, bbxes

bbx_map = {}
abs_idx = 0
with Pool() as pool:
    results = list(tqdm(pool.imap(process_file, all_fps), total=len(all_fps)))

for frame_indices, bbx_indices, bbxes in results:
    for frame_idx, bbx_idx, bbx in zip(frame_indices, bbx_indices, bbxes):
        bbx_map[abs_idx] = (frame_idx, bbx_idx, bbx)
        abs_idx += 1

  0%|          | 0/173647 [00:00<?, ?it/s]

In [3]:
import sys
from pympler import asizeof

def get_size(obj):
    return asizeof.asizeof(obj)

# Assuming bbx_map is your dictionary
size_bytes = get_size(bbx_map)
size_mb = size_bytes / (1024 * 1024)

print(f"Size of bbx_map: {size_mb:.2f} MB")

# Alternative method using sys.getsizeof (less accurate for complex objects)
approx_size_bytes = sys.getsizeof(bbx_map)
approx_size_mb = approx_size_bytes / (1024 * 1024)

print(f"Approximate size of bbx_map: {approx_size_mb:.2f} MB")

Size of bbx_map: 0.00 MB
Approximate size of bbx_map: 0.00 MB
